# Association Rule for Store Dataset

## Load Data

Load data dan tunjukan 5 row pertamanya

Dataset :
https://raw.githubusercontent.com/GTerrence/dataset/main/Groceries%20data.csv

In [1]:
! pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

url = "https://raw.githubusercontent.com/GTerrence/dataset/main/Groceries%20data.csv"
df = pd.read_csv(url)
df.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1808,2015-07-21,tropical fruit,2015,7,21,1
1,2552,2015-05-01,whole milk,2015,5,1,4
2,2300,2015-09-19,pip fruit,2015,9,19,5
3,1187,2015-12-12,other vegetables,2015,12,12,5
4,3037,2015-01-02,whole milk,2015,1,2,4


<h2> EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
 3   year             38765 non-null  int64 
 4   month            38765 non-null  int64 
 5   day              38765 non-null  int64 
 6   day_of_week      38765 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 2.1+ MB


In [4]:
df.describe()

,Member_number,year,month,day,day_of_week
count,38765.000000,38765.000000,38765.000000,38765.000000,38765.000000
mean,3003.641868,2014.528518,6.477570,15.753231,3.014498
std,1153.611031,0.499193,3.431561,8.801391,1.987669
min,1000.000000,2014.000000,1.000000,1.000000,0.000000
25%,2002.000000,2014.000000,4.000000,8.000000,1.000000
50%,3005.000000,2015.000000,6.000000,16.000000,3.000000
75%,4007.000000,2015.000000,9.000000,23.000000,5.000000
max,5000.000000,2015.000000,12.000000,31.000000,6.000000


In [5]:
df.duplicated().sum()

759

In [7]:
df.isna().sum()

Member_number      0
Date               0
itemDescription    0
year               0
month              0
day                0
day_of_week        0
dtype: int64

<h2> Data Preprocessing

Ubahlah dataset di atas menjadi bentuk one hot encoding. Asumsikanlah bahwa pembelian barang yang dilakukan oleh Customer ID yang sama dalam hari yang sama terjadi dalam 1 transaksi yang sama (satu Order ID)

In [8]:
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week,date
0,1808,2015-07-21,tropical fruit,2015,7,21,1,2015-07-21
1,2552,2015-05-01,whole milk,2015,5,1,4,2015-05-01
2,2300,2015-09-19,pip fruit,2015,9,19,5,2015-09-19
3,1187,2015-12-12,other vegetables,2015,12,12,5,2015-12-12
4,3037,2015-01-02,whole milk,2015,1,2,4,2015-01-02


In [9]:
df['member_date'] = list(zip(df['Member_number'], df['date'].dt.date))

#df['member_date'] = df[["Member_number", "Date"]].apply("_".join, axis=1)
#df['member_date'] = df['Member_number'].astype(str) +"_"+ df["Date"]

df['quantity'] = 1
df.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week,date,member_date,quantity
0,1808,2015-07-21,tropical fruit,2015,7,21,1,2015-07-21,"(1808, 2015-07-21)",1
1,2552,2015-05-01,whole milk,2015,5,1,4,2015-05-01,"(2552, 2015-05-01)",1
2,2300,2015-09-19,pip fruit,2015,9,19,5,2015-09-19,"(2300, 2015-09-19)",1
3,1187,2015-12-12,other vegetables,2015,12,12,5,2015-12-12,"(1187, 2015-12-12)",1
4,3037,2015-01-02,whole milk,2015,1,2,4,2015-01-02,"(3037, 2015-01-02)",1


In [10]:
basket = df.groupby(['member_date', 'itemDescription'])['quantity'].count().unstack().fillna(0)

def convert_values(value):
    if value >= 1:
        return 1
    else:
        return 0 
        
basket = basket.applymap(convert_values)
basket

itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
member_date,,,,,,,,,,,,,,,,,,,,,
"(1000, 2014-06-24)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
"(1000, 2015-03-15)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
"(1000, 2015-05-27)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000, 2015-07-24)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000, 2015-11-25)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(4999, 2015-05-16)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
"(4999, 2015-12-26)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(5000, 2014-09-03)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Apriori Algorithm

Gunakanlah algroitma apriori untuk mencari tahu produk mana yang sering dibeli secara bersama-sama (min support = 0.01)

In [11]:
from mlxtend.frequent_patterns import apriori, association_rules

basket_items = apriori(basket, min_support = 0.01, use_colnames = True)
basket_items

,support,itemsets
0,0.021386,(UHT-milk)
1,0.033950,(beef)
2,0.021787,(berries)
3,0.016574,(beverages)
4,0.045312,(bottled beer)
...,...,...
64,0.010559,"(other vegetables, rolls/buns)"
65,0.014837,"(whole milk, other vegetables)"
66,0.013968,"(whole milk, rolls/buns)"
67,0.011629,"(whole milk, soda)"


Berdasarkan hasil di atas, buatlah association rules nya dengan syarat min confidence = 0.1 

In [12]:
association_rules(basket_items, metric = 'confidence', min_threshold = 0.1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(other vegetables),(whole milk),0.122101,0.157923,0.014837,0.121511,0.769430,-0.004446,0.958551
1,(rolls/buns),(whole milk),0.110005,0.157923,0.013968,0.126974,0.804028,-0.003404,0.964550
2,(soda),(whole milk),0.097106,0.157923,0.011629,0.119752,0.758296,-0.003707,0.956636
3,(yogurt),(whole milk),0.085879,0.157923,0.011161,0.129961,0.822940,-0.002401,0.967861
